In [1]:
import pandas as pd
import numpy as np
import numpy as np
from sklearn.decomposition import PCA
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import scale
import matplotlib.gridspec as gridspec
from scipy import stats
import xgboost as xgb
%matplotlib inline

In [222]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [223]:
train['pos_flag'] = np.where(train['Positive_Directional_Movement']>0,1,0)
train['neg_flag'] = np.where(train['Negative_Directional_Movement']<0,1,0)

In [213]:
#train['month'] = np.ceil(np.ceil(train['timestamp']/20.0)%12)
#test['month'] = np.ceil(np.ceil(test['timestamp']/20.0)%12) 

#train['month'] = np.where(train['month']==0,12,train['month'])
#test['month'] = np.where(test['month']==0,12,test['month'])

In [202]:
#train['year'] = np.ceil(train['timestamp']/240.0)
#test['year'] = np.ceil(test['timestamp']/240.0)

In [224]:
train['week_5'] = np.ceil(np.ceil(train['timestamp']/5.0)%4)
test['week_5'] = np.ceil(np.ceil(test['timestamp']/5.0)%4) 

train['week_5'] = np.where(train['week_5']==0,4,train['week_5'])
test['week_5'] = np.where(test['week_5']==0,4,test['week_5'])

In [225]:
train['day_of_week'] = train['timestamp']%5
test['day_of_week'] = test['timestamp']%5

train['day_of_week'] = np.where(train['day_of_week']==0,5,train['day_of_week'])
test['day_of_week'] = np.where(test['day_of_week']==0,5,test['day_of_week'])

In [226]:
test['pos_flag'] = np.where(test['Positive_Directional_Movement']>0,1,0)
test['neg_flag'] = np.where(test['Negative_Directional_Movement']<0,1,0)

In [227]:
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV 
from sklearn.cross_validation import train_test_split, cross_val_score

In [228]:
 params = {
    "objective": "binary:logistic",
    "booster": "gbtree",
    "max_depth":8,
    "eval_metric": "logloss",
    "eta": 0.01,
    "silent": 1,
    'colsample':0.9,
    'subsample':0.9
    }

In [229]:
target = 'Outcome'
num = 'ID'
timestamp = 'timestamp'
#stock = 'Stock_ID'


predictors = [x for x in train.columns if x not in [target,num,timestamp]]
Y = train['Outcome']
X_train, X_val, y_train, y_val = train_test_split(
    train[predictors],Y, train_size=0.80, random_state=10)

In [230]:
dtrain = xgb.DMatrix(X_train,y_train)
dvalid = xgb.DMatrix(X_val, y_val)

In [231]:
watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
gbm = xgb.train(params, dtrain,2000, evals=watchlist,
              early_stopping_rounds=10,verbose_eval=True)

[0]	train-logloss:0.692833	eval-logloss:0.692864
Multiple eval metrics have been passed: 'eval-logloss' will be used for early stopping.

Will train until eval-logloss hasn't improved in 10 rounds.
[1]	train-logloss:0.692522	eval-logloss:0.692582
[2]	train-logloss:0.692227	eval-logloss:0.69232
[3]	train-logloss:0.691926	eval-logloss:0.69205
[4]	train-logloss:0.691628	eval-logloss:0.691783
[5]	train-logloss:0.691339	eval-logloss:0.691521
[6]	train-logloss:0.691062	eval-logloss:0.691275
[7]	train-logloss:0.690781	eval-logloss:0.691026
[8]	train-logloss:0.690505	eval-logloss:0.690783
[9]	train-logloss:0.690239	eval-logloss:0.690543
[10]	train-logloss:0.689972	eval-logloss:0.690304
[11]	train-logloss:0.689713	eval-logloss:0.690072
[12]	train-logloss:0.689462	eval-logloss:0.689852
[13]	train-logloss:0.689207	eval-logloss:0.689628
[14]	train-logloss:0.688959	eval-logloss:0.689411
[15]	train-logloss:0.688716	eval-logloss:0.689197
[16]	train-logloss:0.688479	eval-logloss:0.688989
[17]	train-lo

In [232]:
test['Outcome'] = gbm.predict(xgb.DMatrix(test[predictors]))

In [233]:
dtest = pd.DataFrame()
dtest['ID'] = test['ID']
dtest['Outcome'] = test['Outcome']

In [234]:
dtest.head()

,ID,Outcome
0,1_554,0.502645
1,2_554,0.432654
2,3_554,0.526929
3,4_554,0.166074
4,6_554,0.403115


In [235]:
dtest.to_csv('subm_10_python.csv',index=False)